# Application of Gibbs Sampling to Mixture of Gaussians

## Task
The file `data_ex06.csv` contains data sampled from a mixture of $K = 4$ Gaussians. The standard deviations of the Gaussians are known and
identical, $\sigma_k = \sigma = 0.5$. Reconstruct the unknown parameters and the latent variables:
$$
    p_k, \mu_k, Z_i
$$
where $Z_i$ represents the assignment of each point to one of the mixture components. To perform the inference, use _Gibbs sampling_.


## Gibbs sampling
**Step 1** <br>
Initialize randomly $p_k, \mu_k, Z_i$. Compute $N_k = \sum_i \chi(Z_i=k)$ and $m_k=\sum_i \chi(Z_i=k) x_k$.

**Step 2** <br>
Perform Gibbs sampling iterations:
- Sample $\mu_k$ for all $k$ from a normal with mean:
  $$
    \mu'_k = \left(\frac{\mu_0}{\sigma_0^2}+\frac{m_k}{\sigma^2}\right)\frac{1}{\left(\frac{1}{\sigma_0^2}+\frac{N_k}{\sigma^2}\right)}
  $$
  and standard deviation:
  $$
    \sigma'_k = \left(\frac{1}{\sigma_0^2}+\frac{N_k}{\sigma^2}\right)^{-\frac{1}{2}}
  $$
  Here $\mu_0 = 0$ and $\sigma_0 = 1000$ are the Gaussian prior parameters. Do this for all $k$.

- Sample the $\vec{p}$ from a Dirichlet distribution with parameters $\gamma'_k=\gamma_k+N_k$. Here $\gamma_k=1$ are the Dirichlet prior parameters.

- Sample the $Z_i$ from a categorical distribution:
  $$
    P(Z_i=k) = \frac{p_k\exp\left\{-\frac{(x_k-\mu_k)^2}{2\sigma^2}\right\}}{\sum_k p_k\exp\left\{-\frac{(x_k-\mu_k)^2}{2\sigma^2}\right\}}
  $$

- If $Z_i$ is updated, update $N_k$ and $m_k$, and consequentially $\vec{\mu}', \vec{\sigma}', \vec{\gamma}', \vec{p}'$.

## Mixture of Gaussians (MoG)
Assume to have $N$ 1-dimensional points generated from a Mixture of $D$ Gaussians with parameters $\mu_i, \sigma_i$ for $i=1, \dots, N$. Then, assuming i.i.d. points, the likelihood function can be written as:
$$
  f\left(\vec{x}|\vec{p}, \vec{\mu}, \vec{\sigma}\right) = \prod_{i=1}^N f\left(x_i|\vec{p}, \vec{\mu}, \vec{\sigma}\right) =  
  \prod_{i=1}^N \sum_{k=1}^D p_k \mathcal{N}(x_i|\mu_k, \sigma_k^2)
$$
Given the likelihood, the posterior becomes:
$$
  f\left(\vec{p}, \vec{\mu}, \vec{\sigma}|\vec{x}\right) = \left(\prod_{i=1}^N \sum_{k=1}^D p_k \mathcal{N}(x_i|\mu_k, \sigma_k^2)\right) \times f_{prior}(\vec{p}, \vec{\mu}, \vec{\sigma})
$$

The problem with respect to Metropolis is that the order of operations involved in the computation of the likelihood is $\mathcal{O}(DN)$, which is computationally heavy and makes the problem intractable. 

A strategy to simplify the problem and make the inference task more interesting is to write the generative process in a slightly different form, i.e. as the combination of 2 random processes:
1. extract $Z_i\in\{1, \dots, D\}$ with probability $\{p_1, \dots, p_D\}$
2. extract $x_i$ from the corresponding component of the mixture: $\mathcal{N}(\mu_i, \sigma_i^2)$

The variables $Z_i$ are called _latent variables_ and are unknown. They represent the mixture component from which each point $x_i$ was sampled. Thus, they are distributed according to a categorical distribution:
$$
  Z_i\sim Cat(\vec{p}) \quad \text{i.e.} \quad P(Z_i=k)=p_k
$$
Then the likelihood becomes:
$$
  f\left(x_i|\vec{\mu}, \vec{\sigma}, Z_i\right) = \mathcal{N}(\mu_{Z_i}, \sigma_{Z_i}^2) \iff f\left(x_i|\vec{\mu}, \vec{\sigma}, Z_i=k\right) = \mathcal{N}(\mu_k, \sigma_k^2)
$$
meaning that when $Z_i = k$,  $x_i$ is sampled from a Gaussian with mean $\mu_k$ and variance $\sigma^2$. 

We can also rewrite the posterior using the latent variables:
$$
\begin{align*}
  f\left(\vec{x}|\vec{\mu}, \vec{\sigma}, \vec{Z}\right) &= \prod_{i=1}^N f\left(x_i|\vec{\mu}, \vec{\sigma}, Z_i\right) = \\
  &= \prod_{i=1}^N \mathcal{N}(x_i|\mu_{Z_i}, \sigma_{Z_i}^2) = \\
  &= \prod_{i=1}^N \left(2\pi\sigma_{Z_i}^2\right)^{-\frac{1}{2}} e^{-\frac{1}{2}\frac{(x_i-\mu_{Z_i})^2}{\sigma_{Z_i}^2}} = \\
  &= \prod_{k=1}^D \left(2\pi\sigma_{Z_i}^2\right)^{-\frac{N_k}{2}} e^{-\frac{1}{2}\sum_{i=1}^N \chi(Z_i=k)\frac{(x_i-\mu_k)^2}{\sigma_{k}^2}}
\end{align*}
$$
where $\chi(Z_i=k)=1$ if $x_i$ is such that $Z_i=k$, else $0$. Here $N_k = \sum_i \chi(Z_i=k)$.

Notice that the model with latent variables $Z_i$ allows to:
1. Drastically reduce the computational complexity of computing the likelihood
2. Use conjugate priors, which considerably simplifies the shape of the posterior, enabling Gibbs sampling
   
At this point we can infer all the parameters from the data, assuming for simplicity that $\sigma_k=\sigma=0.5 \, \forall k$. 

### Priors
- The prior of $\mu_k$ is Gaussian with parameters $\mu_0$, $\sigma_0$: 
  $$ 
    \mu_k|\mu_0, \sigma_0 \sim \mathcal{N}(\mu_0, \sigma_0^2)
  $$
  Then:
  $$ 
    f_{prior}(\vec{\mu}|\mu_0, \sigma_0^2)=\prod_{k=1}^D f_{prior}(\mu_k|\mu_0, \sigma_0^2) = (2\pi\sigma_{0})^{-\frac{D}{2}}\prod_{k=1}^D e^{-\frac{(\mu_0-\mu_k)^2}{2\sigma_0^2}}
  $$

- For $\vec{p}$ we may assume a Dirichlet prior, which is the generalization of a Beta, i.e. the conjugate prior for Bernoulli trials:
  $$
    p_{k}|\vec{\gamma}\sim Dir(\vec{\gamma})
  $$ 
  or 
  $$
    f_{prior}(\vec{p}|\vec{\gamma})=(\beta(\vec{\gamma}))^{-1}\prod_{k=1}^D p_{k}^{\gamma_{k}-1}
  $$
  This is the conjugate prior for the multinomial distribution. We can fix the hyperparameters to have a non-informative prior $\gamma_{k}=1$
  (uniform prior). 

- From the assumptions, we know that 
  $$
    f_{prior}(Z_i=k|p_k)=p_k \quad \Longrightarrow \quad f_{prior}(\vec{Z}|\vec{p}) = \prod_{k=1}^D p_k^{N_k} 
  $$

### Posterior
The posterior simply becomes:
$$
f(\vec{\mu},\vec{Z},\vec{p}|\vec{x})= \frac{1}{E(\vec{x})}\times f_{likelihood}(\vec{x}|\vec{\mu},\sigma, \vec{Z})\times f_{prior}(\vec{\mu}|\mu_{0}, \sigma_{0})\times f_{prior}(\vec{Z}|\vec{p})\times f_{prior}(\vec{p}|\vec{\gamma})
$$
where $E(\vec{x})$ is the evidence. 

### Conditional posteriors
We want to sample the above posterior using Gibbs Sampling. To do so, we need to compute the conditional distributions:
$$
\begin{align*}
  f&(\mu_k | \vec{\mu}_{-k},\vec{Z},\vec{p}, \vec{x}) \\
  f&(p_k | \vec{\mu},\vec{Z},\vec{p}_{-k}, \vec{x}) \\
  f&(Z_i | \vec{\mu},\vec{Z}_{-i},\vec{p}, \vec{x}) 
\end{align*}
$$
Even if it looks very difficult, in the computation most terms cancel out. The computation is done using Bayes theorem and marginalization; for semplicity we report here only the final results. 

For $\mu_k$:
$$
  \mu_{k}|\vec{\mu}_{-k},\vec{Z},\vec{p}, \vec{x}\sim \mathcal{N}(\mu_k', \sigma_k'^2)
$$
with
$$
\begin{align*}
  \mu'_k &= \left(\frac{\mu_0}{\sigma_0^2}+\frac{m_k}{\sigma^2}\right)\frac{1}{\left(\frac{1}{\sigma_0^2}+\frac{N_k}{\sigma^2}\right)} \\
  \sigma'_k &= \left(\frac{1}{\sigma_0^2}+\frac{N_k}{\sigma^2}\right)^{-\frac{1}{2}}
\end{align*}  
$$

For $p_k$:
$$
  \vec{p}|\vec{\mu},\vec{Z},\vec{p}_{-k}, \vec{x}\sim Dir(\vec{\gamma'}) 
$$
with
$$
  \vec{\gamma'}=\vec{\gamma}+\vec{N}
$$

Finally, for $Z_i$:
$$
  Z_{i}|\vec{\mu},\vec{Z}_{-i},\vec{p}, \vec{x} \sim Cat(\vec{p}')
$$
with
$$
  {p'}_k \propto p_k e^{-\frac{(x_{i}-\mu_{k})^{2}}{2\sigma^{2}}}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt